In [1]:
import gym
import collections
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
learning_rate = 0.0005
gamma         = 0.98
buffer_limit  = 50000
batch_size    = 32

In [3]:
class ReplayBuffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit) # FIFO
    
    def put(self, transition):
        self.buffer.append(transition) # 버퍼에 샘플 추가
    
    def sample(self, n):
        mini_batch = random.sample(self.buffer, n) # 버퍼에서 batch size만큼 샘플 추출
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [], [], [], [], []
        
        for transition in mini_batch: # 샘플에서 데이터 추출
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])

        return torch.tensor(s_lst, dtype=torch.float), torch.tensor(a_lst), \
               torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch.float), \
               torch.tensor(done_mask_lst) # 텐서 형태로 반환
    
    def size(self):
        return len(self.buffer)

In [4]:
class Qnet(nn.Module):
    def __init__(self):
        super(Qnet, self).__init__()
        self.fc1 = nn.Linear(4, 128) # input : state
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 2) # output : action_values

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
      
    def sample_action(self, obs, epsilon): # E-greedy
        out = self.forward(obs) # action_values
        coin = random.random()
        if coin < epsilon:
            return random.randint(0,1)
        else : 
            return out.argmax().item()

In [5]:
def train(q, q_target, memory, optimizer):
    for i in range(10): # 10개의 샘플
        s,a,r,s_prime,done_mask = memory.sample(batch_size) # 각 샘플에서 데이터 추출

        q_out = q(s) # action_values
        q_a = q_out.gather(1,a) # selected_action_value
        max_q_prime = q_target(s_prime).max(1)[0].unsqueeze(1) # target network 이용
        target = r + gamma * max_q_prime * done_mask
        loss = F.smooth_l1_loss(q_a, target) # loss
        
        optimizer.zero_grad()
        loss.backward() # cal loss grads
        optimizer.step() # update

In [6]:
def main():
    env = gym.make('CartPole-v1')
    q = Qnet() # Learning network
    q_target = Qnet() # Target network
    q_target.load_state_dict(q.state_dict()) # Learning network와 Target nerwork 동기화
    memory = ReplayBuffer() # 버퍼 생성

    print_interval = 20
    score = 0.0  
    optimizer = optim.Adam(q.parameters(), lr=learning_rate) # Learning network parameters만 업데이트

    for n_epi in range(10000): # 10000개의 에피소드
        epsilon = max(0.01, 0.08 - 0.01*(n_epi/200)) #Linear annealing from 8% to 1% Decaying E-greedy
        s = env.reset()
        done = False

        while not done: # 에피소드가 끝나기 전까지
            a = q.sample_action(torch.from_numpy(s).float(), epsilon) # E-greedy      
            s_prime, r, done, info = env.step(a)
            done_mask = 0.0 if done else 1.0
            memory.put((s,a,r/100.0,s_prime, done_mask)) # reward scailing
            s = s_prime

            score += r
            if done:
                break
            
        if memory.size()>2000: # 에피소드 끝나면 업데이트
            train(q, q_target, memory, optimizer)

        if n_epi%print_interval==0 and n_epi!=0:
            q_target.load_state_dict(q.state_dict())
            print("n_episode :{}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(
                                                            n_epi, score/print_interval, memory.size(), epsilon*100))
            score = 0.0
    env.close()

In [7]:
main()

n_episode :20, score : 10.8, n_buffer : 217, eps : 7.9%
n_episode :40, score : 9.6, n_buffer : 408, eps : 7.8%
n_episode :60, score : 9.7, n_buffer : 601, eps : 7.7%
n_episode :80, score : 10.1, n_buffer : 802, eps : 7.6%
n_episode :100, score : 9.3, n_buffer : 989, eps : 7.5%
n_episode :120, score : 9.5, n_buffer : 1179, eps : 7.4%
n_episode :140, score : 9.4, n_buffer : 1367, eps : 7.3%
n_episode :160, score : 9.9, n_buffer : 1565, eps : 7.2%
n_episode :180, score : 9.8, n_buffer : 1762, eps : 7.1%
n_episode :200, score : 9.6, n_buffer : 1953, eps : 7.0%
n_episode :220, score : 10.2, n_buffer : 2157, eps : 6.9%
n_episode :240, score : 10.2, n_buffer : 2360, eps : 6.8%
n_episode :260, score : 10.1, n_buffer : 2562, eps : 6.7%
n_episode :280, score : 10.1, n_buffer : 2763, eps : 6.6%
n_episode :300, score : 10.4, n_buffer : 2971, eps : 6.5%
n_episode :320, score : 10.5, n_buffer : 3181, eps : 6.4%
n_episode :340, score : 11.2, n_buffer : 3404, eps : 6.3%
n_episode :360, score : 18.2, n